In [1]:
import pandas as pd, os, tqdm, numpy as np

c:\Users\gustavo.ubeda\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gustavo.ubeda\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\gustavo.ubeda\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Total Express

In [2]:
path_files = "./total_express/abrangencias"
col_index_dropna = 1

dfs = []
print("Iniciando o processamento de dados da TOTAL\n")
print("\nExtraindo as abrangências:\n")
for file in os.listdir(path_files):
    try:
        df = pd.read_excel(f"{path_files}/{file}")

        df.dropna(subset=df.columns[col_index_dropna], inplace=True)
        df.reset_index(inplace=True, drop=True)
        df.columns = df.iloc[0]
        df = df[1:]
        df.dropna(axis=1, how='all', inplace=True)

        cods_name = file.split(".")[0].split("-")
        df["Cidade-UF-Origem"] = cods_name[0] + "-" + cods_name[1]

        dfs.append(df)
        print(f"[OK] {file}")
    except:
        pass

abrangencias = pd.concat(dfs).reset_index(drop=True)

Iniciando o processamento de dados da TOTAL


Extraindo as abrangências:

[OK] SANTOS-SP.xlsx
[OK] RIBEIRÃO PRETO-SP.xlsx
[OK] SÃO PAULO-SP.xlsx
[OK] CAMPINAS-SP.xlsx
[OK] NITERÓI-RJ.xlsx
[OK] RIO DE JANEIRO-RJ.xlsx
[OK] BELO HORIZONTE-MG.xlsx
[OK] VITÓRIA-ES.xlsx
[OK] SÃO JOSÉ DOS CAMPOS-SP.xlsx


In [3]:
path_files = "./total_express/tabelas_de_frete"
col_index_dropna = 10

dfs = []
print("\nExtraindo as tabelas de frete:\n")
for file in os.listdir(path_files):
    try:
        try:
            df = pd.read_excel(f"{path_files}/{file}", engine='pyxlsb', sheet_name="Tabela_Unificada")
        except:
            df = pd.read_excel(f"{path_files}/{file}", sheet_name="Tabela_Unificada")

        df.dropna(subset=df.columns[col_index_dropna], inplace=True)
        df.reset_index(inplace=True, drop=True)
        df.columns = df.iloc[1]
        df = df[2:]
        df.dropna(axis=1, how='all', inplace=True)

        dfs_ = []
        for cod in df.columns[2:]:
            df_ = df[df.columns[:2]].copy()
            df_["Cod"] = cod
            df_["Frete"] = df[cod]
            dfs_.append(df_)
        dfs_ = pd.concat(dfs_).reset_index(drop=True)

        cods_name = file.split(".")[0].split("-")
        dfs_["Tipo-Envio"] = cods_name[0]
        dfs_["Cidade-UF-Origem"] = cods_name[1] + "-" + cods_name[2]
        dfs_["Key"] = "-".join(cods_name[:3])

        dfs.append(dfs_)

        print(f"[OK] {file}")
    except:
        pass

fretes = pd.concat(dfs).reset_index(drop=True)
fretes = fretes[fretes["Inicial"] != "Adicional"]


Extraindo as tabelas de frete:

[OK] STANDARD-BARUERI-SP.xlsb
[OK] SHIPPING-BELO HORIZONTE-MG.xlsb
[OK] SHIPPING-VITÓRIA-ES.xlsb
[OK] SHIPPING-SÃO PAULO-SP.xls
[OK] SHIPPING-SÃO JOSÉ DOS CAMPOS-SP.xlsb
[OK] SHIPPING-SANTOS-SP.xlsb
[OK] SHIPPING-RIO DE JANEIRO-RJ.xlsb
[OK] SHIPPING-RIBEIRÃO PRETO-SP.xlsb
[OK] SHIPPING-NITERÓI-RJ.xlsb
[OK] SHIPPING-JUIZ DE FORA-MG.xlsb
[OK] SHIPPING-INDAITUBA-SP.xlsb
[OK] SHIPPING-CAMPINAS-SP.xlsb


In [4]:
path_files = "./total_express/faturas"
col_index_dropna = 1
name_faturas = [i for i in os.listdir(path_files) if "." not in i]

print("\nExtraindo as faturas:\n")
dfs = []
for name_fatura in name_faturas:
    for file in os.listdir(f"{path_files}/{name_fatura}"):
        try:
            try:
                df = pd.read_csv(f"{path_files}/{name_fatura}/{file}", sep=";", encoding="latin1", decimal=",")
            except:
                df = pd.read_excel(f"{path_files}/{name_fatura}/{file}")
            df.dropna(subset="ID", inplace=True)
            df.columns = [i.replace(" ", "") for i in df.columns]

            cods_name = file.split(".")[0].split("-")
            df["Tipo-Envio"] = cods_name[0]
            df["Cidade-UF-Origem"] = cods_name[1] + "-" + cods_name[2]
            df["Key"] = "-".join(cods_name[:3])
            df["Fatura-Data"] = name_fatura

            dfs.append(df)
            print(f"[OK] {name_fatura} - {file}")
        except:
            pass

faturas = pd.concat(dfs).reset_index(drop=True)

gcs = []
print("\nRelacionando as faturas as abrangências:\n")
for fat in tqdm.tqdm(faturas.to_dict(orient="records")):
    try:
        gc = abrangencias[
            (abrangencias["Cidade-UF-Origem"] == fat["Cidade-UF-Origem"]) & 
            (fat["CEP"] >= abrangencias["CEP Inicial"]) &
            (fat["CEP"] <= abrangencias["CEP Final"])
            ]["Geografia Comercial"]
        
        gcs.append(gc.values[0])
    except:
        gcs.append(np.nan)

faturas["GC-Destino"] = gcs
faturas["SerieNF"] = faturas["SerieNF"].astype(int, errors="ignore")


Extraindo as faturas:

[OK] QZ1-10-2023 - SHIPPING-BELO HORIZONTE-MG-CT.csv
[OK] QZ1-10-2023 - SHIPPING-NITERÓI-RJ-CT.csv
[OK] QZ1-10-2023 - SHIPPING-SÃO PAULO-SP-CT.csv
[OK] QZ1-10-2023 - SHIPPING-SÃO JOSÉ DOS CAMPOS-SP-CT.csv
[OK] QZ1-10-2023 - SHIPPING-CAMPINAS-SP-CT.csv
[OK] QZ1-10-2023 - SHIPPING-SANTOS-SP-CT.csv
[OK] QZ1-10-2023 - SHIPPING-VITÓRIA-ES-CT.csv
[OK] QZ2-10-2023 - SHIPPING-CAMPINAS-SP-CT.xlsx
[OK] QZ2-10-2023 - SHIPPING-VITÓRIA-ES-CT.xlsx
[OK] QZ2-10-2023 - SHIPPING-SANTOS-SP-CT.csv
[OK] QZ2-10-2023 - SHIPPING-NITERÓI-RJ-CT.xlsx
[OK] QZ2-10-2023 - SHIPPING-BELO HORIZONTE-MG-CT.xlsx
[OK] QZ2-10-2023 - STANDARD-BARUERI-SP.csv

Relacionando as faturas as abrangências:



100%|██████████| 865/865 [01:21<00:00, 10.61it/s]


## Conferências

In [5]:
conf_faturas = []
conf_faturas.append(faturas.groupby(["NotaFiscal", "SerieNF"])[["Cidade-UF-Origem", "Fatura-Data", "Peso", "CEP", "GC-Destino", "Frete", "Key"]].first())
conf_faturas = pd.concat(conf_faturas, axis=1).sort_values("Frete")
conf_faturas.index.names = ["nf_numero", "serie_nf"]

In [36]:
_itens = pd.read_excel("./dados/group_itens.xlsx", index_col=[0,1])
conferencia = conf_faturas.join(_itens)
conferencia.sort_values("volumes").dropna(subset="volumes")
conferencia = conferencia.sort_values("volumes")

In [37]:
fretes_real = []
print("\nCalculando o frete real:\n")
for conf_ in tqdm.tqdm(conferencia.to_dict(orient="records")):
    try:
        frete_real = fretes[
            (fretes["Key"] == conf_["Key"]) & 
            (fretes["Cod"] == conf_["GC-Destino"]) & 
            (conf_["pesos_cubados_real"] >= fretes["Inicial"]) &
            (conf_["pesos_cubados_real"] <= fretes["Final"])
            ]["Frete"]
        
        fretes_real.append(frete_real.values[0])
    except:
        fretes_real.append(np.nan)

conferencia["Frete real"] = fretes_real
conferencia["Frete-Dif"] = conferencia["Frete"] - conferencia["Frete real"]


Calculando o frete real:



  0%|          | 0/831 [00:00<?, ?it/s]

100%|██████████| 831/831 [00:06<00:00, 131.66it/s]


In [38]:
list_fat_names = os.listdir("./total_express/conferencias")

for name_fatura in name_faturas:
    fat_name = f"{name_fatura}.xlsx"
    if "[OK] " + fat_name not in list_fat_names:
        conferencia[conferencia["Fatura-Data"]==name_fatura].to_excel(f"./total_express/conferencias/{fat_name}")